# Task 2: Encrypting a Message (Encapsulation & Symmetric Encryption)

**Estimated Time: 15-20 minutes**

## Learning Objectives

Upon completing this task, you should be able to:

1.  **Understand the role of the sender** in a public-key cryptosystem.
2.  **Apply** the concept of **Key Encapsulation (KEM)** using ML-KEM and a recipient's public key to securely generate a shared secret.
3.  **Recognize** that the KEM process produces two main outputs: an **encapsulated key (ciphertext)** to be sent to the recipient, and the **shared secret** itself (kept by the sender for immediate use).
4.  **Understand** how the KEM-derived shared secret is used as a key for a **symmetric cipher** (like AES-GCM) to encrypt the actual message.
5.  **Identify** the components needed for AES-GCM encryption: a key and a nonce (Number used ONCE).
6.  **Interact** with the Python script to perform encapsulation and symmetric encryption.
7.  **Prepare** the necessary outputs (encapsulated key, nonce, encrypted message) for the recipient (Task 3).

## Understanding the Task and the Code

In this "sender" role, your goal is to encrypt a message so that only the recipient (who has the corresponding private key from Task 1) can decrypt it. We achieve this using a hybrid approach:

1.  **Key Encapsulation (ML-KEM):** You'll use the recipient's **public key** (from Task 1) to generate a fresh, random symmetric key (the "shared secret"). Simultaneously, ML-KEM "encapsulates" this shared secret. This encapsulation process creates a ciphertext (let's call it `ciphertext_kem` or "encapsulated key") that can only be "decapsulated" (opened) by the recipient using their private key to retrieve the same shared secret.
2.  **Symmetric Encryption (AES-GCM):** Once you have the `shared_secret_kem`, you'll use it as the key for a highly efficient and secure symmetric encryption algorithm called AES-GCM (Advanced Encryption Standard in Galois/Counter Mode). AES-GCM will encrypt your actual plaintext message. AES-GCM also requires a "nonce" (a number used once), which must be unique for every message encrypted with the same key.

**Encryption Process (`on_encrypt_button_clicked` function):**

*   **Input Validation:** Checks if an algorithm is selected, a public key is provided, and a message is entered.
*   **Decode Public Key:** The Base64 encoded public key from Task 1 is decoded back into its raw byte format.
*   **KEM Encapsulation:**
    *   `with oqs.KeyEncapsulation(selected_kem_alg) as kem:`: Initializes the KEM.
    *   `ciphertext_kem, shared_secret_kem = kem.encap_secret(public_key_bytes)`: This is the core KEM operation.
        *   `public_key_bytes`: The recipient's public key.
        *   `ciphertext_kem`: The encapsulated shared secret. This is what you'll send to the recipient.
        *   `shared_secret_kem`: The actual shared secret (a 32-byte value for Kyber/ML-KEM). This is *not* sent directly but used immediately to encrypt the message.
*   **Symmetric Encryption (AES-256-GCM):**
    *   `aes_key = shared_secret_kem`: The shared secret from ML-KEM is used as the AES key.
    *   `nonce = os.urandom(12)`: Generates a random 12-byte nonce. This nonce is critical for AES-GCM's security and must also be sent to the recipient (unencrypted).
    *   `aesgcm = AESGCM(aes_key)`: Initializes the AES-GCM cipher.
    *   `encrypted_message_bytes = aesgcm.encrypt(nonce, plaintext_bytes, None)`: Encrypts the plaintext message.
*   **Prepare Outputs:** The `ciphertext_kem`, `nonce`, and `encrypted_message_bytes` are all Base64 encoded for easy display and copying. These are then shown in their respective output text areas.

**Your Interaction:**

1.  Run the Python code cell below.
2.  From the **ML-KEM Algorithm** dropdown, select the *exact same algorithm variant* that you (or the "recipient") used in Task 1 to generate the key pair.
3.  Go back to your Task 1 output. Carefully copy the entire **Base64 encoded Public Key**.
4.  Paste this Public Key into the "Recipient's Public Key (Base64)" text area in Task 2.
5.  Type a message you want to encrypt in the "Plaintext Message" area.
6.  Click the "Encrypt Message" button.
7.  Observe the three Base64 encoded outputs: "Encapsulated Key," "Nonce," and "Encrypted Message."
8.  Read the status messages to follow the steps the code is performing.

## Next Steps

The three Base64 encoded strings generated in this task (`Encapsulated Key`, `Nonce`, and `Encrypted Message`) are what you would typically transmit to the recipient over an insecure channel. In **Task 3**, you will play the role of the recipient and use these three pieces of information, along with the **private key** from Task 1, to decrypt the message.

Make sure to copy these three values accurately for Task 3!

---
**Now, run the code cell below to perform Task 2.**

In [1]:
import oqs
import ipywidgets as widgets
from IPython.display import display, HTML as IPHTML
import base64
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.backends import default_backend
import os

# --- KEM Algorithms (should match those offered in Task 1) ---
supported_kems_task2 = ['ML-KEM-512', 'ML-KEM-768', 'ML-KEM-1024']

# --- UI Elements ---
header_task2 = IPHTML("<h2>Task 2: Encrypt a Message (Encapsulation & Symmetric Encryption)</h2>")
description_task2 = IPHTML("""
<p>In this task, you will act as the <b>sender</b>. You will use the <b>Public Key</b> (generated in Task 1 by the recipient) to securely establish a shared secret.
This shared secret will then be used with a symmetric cipher (AES-256-GCM) to encrypt your message.</p>
<ol>
    <li>Select the <em>same ML-KEM algorithm</em> that was used to generate the keys in Task 1.</li>
    <li>Paste the <b>Base64 encoded Public Key</b> (copied from Task 1) into the designated text area.</li>
    <li>Type the message you want to encrypt.</li>
    <li>Click 'Encrypt Message'.</li>
</ol>
<p>The outputs (Encapsulated Key, Nonce, and Encrypted Message) will be needed in Task 3 for decryption.</p>
""")

kem_alg_dropdown_task2 = widgets.Dropdown(
    options=supported_kems_task2,
    value=supported_kems_task2[1] if len(supported_kems_task2) > 1 else supported_kems_task2[0], # Default to ML-KEM-768
    description='ML-KEM Algorithm:',
    disabled=False,
    style={'description_width': 'initial'}
)

public_key_b64_input_task2 = widgets.Textarea(
    value='',
    placeholder='Paste the Base64 encoded PUBLIC KEY from Task 1 here.',
    description='Recipient\'s Public Key (Base64):',
    layout={'width': '95%', 'height': '100px'},
    disabled=False,
    style={'description_width': 'initial'}
)

message_to_encrypt_input_task2 = widgets.Textarea(
    value='Hello, Post-Quantum World!',
    placeholder='Type your secret message here.',
    description='Plaintext Message:',
    layout={'width': '95%', 'height': '80px'},
    disabled=False,
    style={'description_width': 'initial'}
)

encrypt_button_task2 = widgets.Button(
    description='Encrypt Message',
    button_style='primary',
    tooltip='Encapsulate a shared secret and encrypt the message',
    icon='lock'
)

# Outputs for Task 3
encapsulated_key_output_task2 = widgets.Textarea(
    value='',
    placeholder='Encapsulated Key (Ciphertext from KEM) will appear here (Base64). Copy for Task 3.',
    description='Encapsulated Key (for Task 3):',
    layout={'width': '95%', 'height': '100px'},
    disabled=True,
    style={'description_width': 'initial'}
)

nonce_output_task2 = widgets.Textarea(
    value='',
    placeholder='AES-GCM Nonce will appear here (Base64). Copy for Task 3.',
    description='Nonce (for Task 3):',
    layout={'width': '95%', 'height': '60px'},
    disabled=True,
    style={'description_width': 'initial'}
)

encrypted_message_output_task2 = widgets.Textarea(
    value='',
    placeholder='Symmetrically Encrypted Message will appear here (Base64). Copy for Task 3.',
    description='Encrypted Message (for Task 3):',
    layout={'width': '95%', 'height': '100px'},
    disabled=True,
    style={'description_width': 'initial'}
)

status_output_task2 = widgets.Output(layout={'width': '95%'})

next_steps_info_task2 = widgets.HTML(value="""
    <hr>
    <h4>Next Steps:</h4>
    <p>The <b>Encapsulated Key</b>, <b>Nonce</b>, and <b>Encrypted Message</b> displayed above (all Base64 encoded) are what you would send to the recipient.</p>
    <p>You will manually copy these three pieces of information and paste them into <b>Task 3 (Decryption/Decapsulation)</b>.</p>
    <p>Ensure you copy each entire Base64 string accurately.</p>
    """,
    layout={'width': '95%'}
)

_task2_outputs = {
    "encapsulated_key_bytes": None,
    "nonce_bytes": None,
    "encrypted_message_bytes": None,
    "algorithm": None
}


# --- Button Click Handler ---
def on_encrypt_button_clicked(b):
    selected_kem_alg = kem_alg_dropdown_task2.value
    public_key_b64 = public_key_b64_input_task2.value
    plaintext_message_str = message_to_encrypt_input_task2.value

    # Clear previous outputs
    encapsulated_key_output_task2.value = ""
    nonce_output_task2.value = ""
    encrypted_message_output_task2.value = ""
    _task2_outputs["encapsulated_key_bytes"] = None
    _task2_outputs["nonce_bytes"] = None
    _task2_outputs["encrypted_message_bytes"] = None
    _task2_outputs["algorithm"] = None


    with status_output_task2:
        status_output_task2.clear_output()

        if not selected_kem_alg:
            print("Error: Please select an ML-KEM algorithm.")
            return
        if not public_key_b64.strip():
            print("Error: Please paste the recipient's Base64 encoded Public Key from Task 1.")
            return
        if not plaintext_message_str.strip():
            print("Error: Please enter a message to encrypt.")
            return

        print(f"Starting encryption process with {selected_kem_alg}...")

        try:
            # 1. Decode the Base64 public key
            print("Step 1: Decoding Base64 public key...")
            public_key_bytes = base64.b64decode(public_key_b64)
            print(f"  Public key decoded successfully ({len(public_key_bytes)} bytes).")

            # 2. KEM Encapsulation: Generate a shared secret and encapsulate it for the recipient
            print(f"\nStep 2: Encapsulating shared secret using {selected_kem_alg} with the provided public key...")
            with oqs.KeyEncapsulation(selected_kem_alg) as kem:
                # `encap_secret` takes the recipient's public key and returns:
                # - ciphertext_kem: The encapsulated shared secret (to be sent to the recipient)
                # - shared_secret_kem: The actual shared secret (symmetric key for AES)
                ciphertext_kem, shared_secret_kem = kem.encap_secret(public_key_bytes)
            
            print(f"  KEM encapsulation successful.")
            print(f"    Generated shared secret length: {len(shared_secret_kem)} bytes.")
            print(f"    Encapsulated key (ciphertext_kem) length: {len(ciphertext_kem)} bytes.")

            # KEM shared secrets from Kyber are 32 bytes, suitable for AES-256.
            if len(shared_secret_kem) != 32:
                # This should not happen with standard Kyber variants.
                print(f"Warning: KEM shared secret length is {len(shared_secret_kem)} bytes, expected 32 for AES-256. Truncating/padding might be insecure. Check KEM.")
                # Forcing it to 32 bytes for AES256 (not generally recommended without a KDF)
                aes_key = (shared_secret_kem + b'\0' * 32)[:32]
            else:
                aes_key = shared_secret_kem
            
            # 3. Symmetric Encryption (AES-256-GCM)
            print(f"\nStep 3: Symmetrically encrypting the message using AES-256-GCM...")
            plaintext_bytes = plaintext_message_str.encode('utf-8')
            
            # Generate a random nonce (IV) for AES-GCM. 12 bytes is common.
            nonce = os.urandom(12)
            print(f"  Generated AES-GCM Nonce: {len(nonce)} bytes.")

            aesgcm = AESGCM(aes_key)
            encrypted_message_bytes = aesgcm.encrypt(nonce, plaintext_bytes, None) # No additional authenticated data
            print(f"  Message encrypted successfully. Ciphertext length: {len(encrypted_message_bytes)} bytes.")

            # 4. Prepare outputs for Task 3 (Base64 encode them)
            print(f"\nStep 4: Encoding outputs in Base64 for Task 3...")
            ciphertext_kem_b64 = base64.b64encode(ciphertext_kem).decode('utf-8')
            nonce_b64 = base64.b64encode(nonce).decode('utf-8')
            encrypted_message_b64 = base64.b64encode(encrypted_message_bytes).decode('utf-8')

            encapsulated_key_output_task2.value = ciphertext_kem_b64
            nonce_output_task2.value = nonce_b64
            encrypted_message_output_task2.value = encrypted_message_b64

            # Store raw bytes as well, just in case, though lab flow is copy-paste
            _task2_outputs["encapsulated_key_bytes"] = ciphertext_kem
            _task2_outputs["nonce_bytes"] = nonce
            _task2_outputs["encrypted_message_bytes"] = encrypted_message_bytes
            _task2_outputs["algorithm"] = selected_kem_alg


            print("\nEncryption complete! The following Base64 encoded items are ready for Task 3:")
            print("  1. Encapsulated Key (Ciphertext from KEM)")
            print("  2. Nonce (for AES-GCM)")
            print("  3. Encrypted Message (AES-GCM ciphertext)")
            print("Please copy these from the text areas above.")

        except base64.binascii.Error as e:
            print(f"Error decoding Base64 Public Key: {e}. Please ensure it's a valid Base64 string from Task 1.")
        except oqs.MechanismNotSupportedError:
            print(f"Error: The KEM algorithm '{selected_kem_alg}' is not supported by your liboqs build or does not match the public key.")
        except oqs.OpenSSLError as e: # Can be raised if public key is invalid for the KEM
            print(f"OQS Error (often due to mismatched public key or corrupted key): {e}")
            print(f"Make sure the KEM algorithm selected here ({selected_kem_alg}) MATCHES the one used to generate the public key in Task 1.")
        except Exception as e:
            print(f"An unexpected error occurred during encryption: {e}")
            import traceback
            traceback.print_exc()

encrypt_button_task2.on_click(on_encrypt_button_clicked)

# --- Display UI ---
display(header_task2)
display(description_task2)

main_ui_container_task2 = widgets.VBox([
    kem_alg_dropdown_task2,
    public_key_b64_input_task2,
    message_to_encrypt_input_task2,
    encrypt_button_task2,
    widgets.HTML("<hr><h3>Outputs for Task 3 (Copy these):</h3>"), # Separator
    encapsulated_key_output_task2,
    nonce_output_task2,
    encrypted_message_output_task2,
    status_output_task2,
    next_steps_info_task2
], layout={'width': '100%'})

display(main_ui_container_task2)

/usr/local/lib/python3.13/site-packages/oqs/__init__.py:1: UserWarning: liboqs version (major, minor) 0.13.1-dev differs from liboqs-python version 0.12.0
  from oqs.oqs import (
